In [56]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 29.7633,
	"longitude": -95.3633,
	"start_date": "2020-01-31",
	"end_date": "2025-01-31",
	"hourly": ["temperature_2m", "rain"],
	"timezone": "America/Chicago"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_rain = hourly.Variables(1).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["Farenheit_Temperature"] = hourly_temperature_2m
hourly_data["Inches_Rain"] = hourly_rain

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(display(hourly_dataframe))

hourly_dataframe.to_csv("/home/ctand/EnergyVision/data/raw/weather.csv", index=False)



Coordinates 29.771528244018555°N -95.33099365234375°E
Elevation 14.0 m asl
Timezone b'America/Chicago' b'GMT-6'
Timezone difference to GMT+0 -21600 s


,date,Farenheit_Temperature,Inches_Rain
0,2020-01-31 06:00:00+00:00,9.906500,0.0
1,2020-01-31 07:00:00+00:00,10.006500,0.0
2,2020-01-31 08:00:00+00:00,10.006500,0.0
3,2020-01-31 09:00:00+00:00,9.656500,0.0
4,2020-01-31 10:00:00+00:00,8.356501,0.0
...,...,...,...
43867,2025-02-01 01:00:00+00:00,15.256500,0.0
43868,2025-02-01 02:00:00+00:00,13.956500,0.0
43869,2025-02-01 03:00:00+00:00,13.256500,0.0
43870,2025-02-01 04:00:00+00:00,12.506500,0.0


None
